# Machine Learning Zoo - Comprehensive Guide

A hands-on walkthrough of every major subsystem in the Machine Learning Zoo: model composition, training strategies, reinforcement learning, neuro-symbolic reasoning, fairness, explainability, and more.

**Table of Contents**

1. [Setup & Imports](#1)
2. [Device Management](#2)
3. [The Registry System](#3)
4. [Backbones (Feature Extractors)](#4)
5. [Heads (Task-Specific Layers)](#5)
6. [Model Composition with `build_model`](#6)
7. [Evaluation Toolkit](#7)
8. [Neuro-Symbolic Methods](#8)
9. [Reinforcement Learning - Single Agent](#9)
10. [Multi-Agent RL Environments](#10)
11. [Multi-Agent Policy & MAPPO Training](#11)
12. [Domain Adaptation](#12)
13. [Continual Learning (EWC)](#13)
14. [Federated Learning](#14)
15. [Fairness Auditing](#15)
16. [Explainability](#16)
17. [Sidecar & IPC Architecture](#17)
18. [Configuration with Hydra / OmegaConf](#18)
19. [Project Architecture](#19)
20. [Next Steps & Resources](#20)

<a id="1"></a>
## 1. Setup & Imports

Add the project root to `sys.path` so we can import `src` modules directly from the notebook.

In [ ]:
import sys, pathlib

# Ensure the project root is on the import path
ROOT = pathlib.Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import torch
import numpy as np

print(f"PyTorch {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available:  {hasattr(torch.backends, 'mps') and torch.backends.mps.is_available()}")

<a id="2"></a>
## 2. Device Management

`DeviceManager` probes the system for CPUs, NVIDIA GPUs, and Apple MPS, then recommends the best device for a given workload.

In [ ]:
from src.device.manager import DeviceManager

dm = DeviceManager()
system_info = dm.probe()

print(f"Platform:  {system_info.platform}")
print(f"CPU:       {system_info.cpu.brand} ({system_info.cpu.cores_physical}P / {system_info.cpu.cores_logical}L cores)")
print(f"RAM:       {system_info.ram_total_mb:,} MB total, {system_info.ram_available_mb:,} MB free")
print(f"GPUs:      {len(system_info.gpus)}")
for gpu in system_info.gpus:
    print(f"  [{gpu.index}] {gpu.name} ({gpu.vendor}) - {gpu.vram_total_mb:,} MB VRAM")

recommended = dm.best_device_for("training", model_size_mb=500)
print(f"\nRecommended device for training (500 MB model): {recommended}")

<a id="3"></a>
## 3. The Registry System

Every backbone and head is registered at import time via decorators (`@register_backbone`, `@register_head`). You can list available components and look them up by name.

In [ ]:
from src.models.backbones import BACKBONE_REGISTRY
from src.models.heads import HEAD_REGISTRY

print("Available backbones:")
for name, cls in BACKBONE_REGISTRY.items():
    print(f"  {name:20s} -> {cls.__name__}")

print("\nAvailable heads:")
for name, cls in HEAD_REGISTRY.items():
    print(f"  {name:20s} -> {cls.__name__}")

<a id="4"></a>
## 4. Backbones (Feature Extractors)

Backbones are task-agnostic networks that turn raw inputs into fixed-size feature vectors. The Zoo ships with Transformer, LSTM, Mamba, Conv, HuggingFace, Vision, and MultiModal backbones.

In [ ]:
from src.models.backbones import (
    TransformerBackbone, TransformerBackboneConfig,
    LSTMBackbone, LSTMBackboneConfig,
)

# --- Transformer backbone ---
tf_cfg = TransformerBackboneConfig(
    input_dim=16, hidden_dim=64, num_layers=2, num_heads=4, ff_dim=128, dropout=0.1,
)
tf_backbone = TransformerBackbone(tf_cfg)

x_seq = torch.randn(2, 10, 16)                     # (batch=2, seq=10, features=16)
tf_out = tf_backbone(x_seq)
print(f"Transformer  input:  {tuple(x_seq.shape)}")
print(f"Transformer  output: {tuple(tf_out.shape)}  (output_dim={tf_backbone.output_dim})")

# --- LSTM backbone ---
lstm_cfg = LSTMBackboneConfig(
    input_dim=16, hidden_dim=64, num_layers=2, bidirectional=True, dropout=0.1,
)
lstm_backbone = LSTMBackbone(lstm_cfg)

lstm_out = lstm_backbone(x_seq)
print(f"\nLSTM  input:  {tuple(x_seq.shape)}")
print(f"LSTM  output: {tuple(lstm_out.shape)}  (output_dim={lstm_backbone.output_dim})")

<a id="5"></a>
## 5. Heads (Task-Specific Layers)

Heads attach to backbone features and produce task-specific outputs: class logits, regression values, RL policies, etc.

In [ ]:
from src.models.heads import (
    ClassificationHead, ClassificationHeadConfig,
    RegressionHead, RegressionHeadConfig,
    RLPolicyHead, RLPolicyHeadConfig,
)

features = torch.randn(2, 64)  # pretend backbone output (batch=2, dim=64)

# --- Classification ---
cls_head = ClassificationHead(ClassificationHeadConfig(
    input_dim=64, num_classes=5, hidden_dims=(32,), pool_type="mean",
))
logits = cls_head(features)
print(f"Classification logits: {tuple(logits.shape)}")  # (2, 5)

# --- Regression ---
reg_head = RegressionHead(RegressionHeadConfig(
    input_dim=64, output_dim=1, output_activation="none",
))
preds = reg_head(features)
print(f"Regression output:     {tuple(preds.shape)}")  # (2, 1)

# --- RL Policy (Actor-Critic) ---
rl_head = RLPolicyHead(RLPolicyHeadConfig(
    input_dim=64, action_dim=3, continuous=False, hidden_dims=(32,),
))
policy_out = rl_head(features)
print(f"\nRL Policy output fields: {policy_out._fields}")
print(f"  action_logits: {tuple(policy_out.action_logits.shape)}")
print(f"  value:         {tuple(policy_out.value.shape)}")

<a id="6"></a>
## 6. Model Composition with `build_model`

`build_model` is the primary factory: pick a backbone name, a head name, pass config dicts, and get a ready-to-train `ComposedModel`.

In [ ]:
from src.models.composed import build_model

# Build a Transformer + Classification model in one call
model = build_model(
    backbone_name="transformer",
    head_name="classification",
    backbone_config={"input_dim": 16, "hidden_dim": 64, "num_layers": 2, "num_heads": 4},
    head_config={"num_classes": 5},
)

x = torch.randn(4, 10, 16)     # (batch=4, seq=10, features=16)
out = model(x)
print(f"ComposedModel output: {tuple(out.shape)}")  # (4, 5)

# Parameter count
total_params = sum(p.numel() for p in model.parameters())
trainable   = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parameters: {total_params:,} total, {trainable:,} trainable")

# Freeze the backbone for transfer learning
model.freeze_backbone()
trainable_after = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"After freeze_backbone: {trainable_after:,} trainable (head only)")

<a id="7"></a>
## 7. Evaluation Toolkit

`Evaluator` provides unified metrics for classification (accuracy, macro-F1), regression (MSE, MAE), and generation (perplexity).

In [ ]:
from src.training.evaluation import Evaluator

# Classification metrics
y_true = torch.tensor([0, 1, 2, 1, 0, 2, 1, 0])
y_pred = torch.tensor([0, 1, 2, 1, 0, 1, 1, 0])  # one mistake
cls_metrics = Evaluator.evaluate("classification", y_true, y_pred)
print("Classification:", cls_metrics)

# Regression metrics
y_true_r = torch.tensor([1.0, 2.0, 3.0, 4.0])
y_pred_r = torch.tensor([1.1, 2.2, 2.8, 4.3])
reg_metrics = Evaluator.evaluate("regression", y_true_r, y_pred_r)
print("Regression:    ", reg_metrics)

<a id="8"></a>
## 8. Neuro-Symbolic Methods

`NeuroSymbolicNetwork` fuses a neural pathway with symbolic reasoning (learnable rules + a differentiable logic program executor). Three integration modes are supported: **gated**, **residual**, and **attention**.

In [ ]:
from src.models.neuro_symbolic import NeuroSymbolicConfig, NeuroSymbolicNetwork

ns_cfg = NeuroSymbolicConfig(
    input_dim=32, hidden_dim=64, output_dim=5,
    num_rules=16, rule_dim=32, num_predicates=8,
    integration_mode="gated", symbolic_depth=2,
)
ns_model = NeuroSymbolicNetwork(ns_cfg)

x = torch.randn(4, 32)  # (batch=4, input_dim=32)
ns_out = ns_model(x)

print("NeuroSymbolicOutput fields:", ns_out._fields)
print(f"  prediction:     {tuple(ns_out.prediction.shape)}")
print(f"  neural_output:  {tuple(ns_out.neural_output.shape)}")
print(f"  symbolic_output:{tuple(ns_out.symbolic_output.shape)}")
print(f"  rule_attention: {tuple(ns_out.rule_attention.shape)}")
print(f"  confidence:     {tuple(ns_out.confidence.shape)}")

# Symbolic constraint loss: classes 0 and 1 are mutually exclusive,
# class 2 implies class 3
constraints = [(0, 1, "mutex"), (2, 3, "implies")]
penalty = ns_model.symbolic_loss(ns_out.prediction, constraints)
print(f"\nSymbolic constraint penalty: {penalty.item():.4f}")

# Inspect learned rule importance
rules = ns_model.get_rule_importance()
print(f"Rule embeddings shape: {tuple(rules.shape)}")

<a id="9"></a>
## 9. Reinforcement Learning - Single Agent

The `TradingEnv` is a Gymnasium-compatible environment where an RL agent trades a price series. We pair it with an `RLPolicyHead` and run a short episode.

In [ ]:
from src.envs import TradingEnv

env = TradingEnv(initial_capital=10_000.0, lookback=10, max_steps=50)
obs, info = env.reset(seed=42)
print(f"Observation shape: {obs.shape}")   # (lookback, 6)
print(f"Action space:      {env.action_space}")  # Discrete(3)

# Run a short episode with the RL policy head
flat_dim = obs.shape[0] * obs.shape[1]   # flatten obs for the head
rl_head = RLPolicyHead(RLPolicyHeadConfig(
    input_dim=flat_dim, action_dim=3, continuous=False, hidden_dims=(64,),
))

total_reward = 0.0
for step in range(20):
    obs_t = torch.tensor(obs.flatten(), dtype=torch.float32).unsqueeze(0)
    action, log_prob, value = rl_head.get_action(obs_t)
    obs, reward, terminated, truncated, info = env.step(int(action.item()))
    total_reward += reward
    if terminated or truncated:
        break

print(f"\nEpisode finished after {step+1} steps")
print(f"Total reward: {total_reward:.4f}")
print(f"Final portfolio: ${info['portfolio_value']:.2f}")

<a id="10"></a>
## 10. Multi-Agent RL Environments

Two built-in multi-agent environments demonstrate cooperative and competitive scenarios:
- **CooperativeGatheringEnv** -- agents collect resources on a grid with shared rewards
- **CompetitiveArenaEnv** -- agents compete for territory control

In [ ]:
from src.envs.multi_agent import (
    MultiAgentEnvConfig, CooperativeGatheringEnv, CompetitiveArenaEnv,
)

# --- Cooperative gathering ---
coop_cfg = MultiAgentEnvConfig(num_agents=3, grid_size=8, max_steps=50, reward_type="shared")
coop_env = CooperativeGatheringEnv(coop_cfg, num_resources=6)

obs, _ = coop_env.reset(seed=0)
print(f"Agents: {coop_env.agents}")
print(f"Obs shape per agent: {obs['agent_0'].shape}")

# Random rollout
for t in range(20):
    actions = {a: coop_env._agent_act_space.sample() for a in coop_env.agents}
    obs, rewards, done, truncated, info = coop_env.step(actions)
    if done or truncated:
        break

print(f"After {t+1} steps -> collected {info['collected']}/{info['total_resources']} resources")
print(f"Rewards: { {a: f'{r:.2f}' for a, r in rewards.items()} }")

# --- Competitive arena ---
comp_cfg = MultiAgentEnvConfig(num_agents=3, grid_size=6, max_steps=30, reward_type="individual")
comp_env = CompetitiveArenaEnv(comp_cfg)

obs, _ = comp_env.reset(seed=0)
for t in range(15):
    actions = {a: comp_env._agent_act_space.sample() for a in comp_env.agents}
    obs, rewards, done, truncated, info = comp_env.step(actions)

print(f"\nCompetitive arena after {t+1} steps:")
print(f"  Territory: {info['territory']}")
print(f"  Unclaimed: {info['unclaimed']}")

<a id="11"></a>
## 11. Multi-Agent Policy & MAPPO Training

`MultiAgentPolicyHead` implements centralized-training-decentralized-execution (CTDE): agents share a centralized value function during training but act independently at inference time. `MAPPOTrainer` orchestrates the full training loop.

In [ ]:
from src.models.heads import MultiAgentPolicyHead, MultiAgentPolicyHeadConfig
from src.training.multi_agent_rl import MARLConfig, MAPPOTrainer

num_agents = 3
obs_dim = obs["agent_0"].shape[0]  # from the competitive env above

# Build multi-agent policy
ma_head = MultiAgentPolicyHead(MultiAgentPolicyHeadConfig(
    input_dim=obs_dim,
    action_dim=5,          # 5 movement actions
    num_agents=num_agents,
    continuous=False,
    shared_parameters=True,
    hidden_dims=(64,),
))

# Quick forward pass
dummy_features = torch.randn(num_agents, 2, obs_dim)  # (agents, batch=2, obs_dim)
ma_out = ma_head(dummy_features)
print(f"action_logits: {tuple(ma_out.action_logits.shape)}")  # (agents, batch, 5)
print(f"values:        {tuple(ma_out.values.shape)}")          # (agents, batch)

# Set up MAPPO trainer
marl_cfg = MARLConfig(
    num_agents=num_agents,
    learning_rate=3e-4,
    gamma=0.99,
    rollout_length=32,
    num_epochs=2,
    num_minibatches=2,
)
trainer = MAPPOTrainer(policy=ma_head, config=marl_cfg)

# Collect one rollout and update
comp_env2 = CompetitiveArenaEnv(comp_cfg)
obs_init, _ = comp_env2.reset(seed=1)
rollout_info = trainer.collect_rollout(comp_env2, obs_init)
metrics = trainer.update()

print(f"\nMAPPO update metrics:")
for k, v in metrics.items():
    print(f"  {k}: {v:.4f}")
print(f"Total env steps: {rollout_info['total_steps']}")

<a id="12"></a>
## 12. Domain Adaptation

When training and deployment data come from different distributions, domain adaptation helps bridge the gap. The Zoo provides:

| Component | Purpose |
|---|---|
| `GradientReversalLayer` | Reverses gradients so the feature extractor learns domain-invariant representations |
| `MMDLoss` | Maximum Mean Discrepancy -- measures distribution distance in an RKHS |
| `DomainDiscriminator` | Binary classifier distinguishing source vs. target domains |

In [ ]:
from src.training.domain_adaptation import GradientReversalLayer, MMDLoss, DomainDiscriminator

# Synthetic source and target distributions
source_features = torch.randn(32, 64)          # source domain
target_features = torch.randn(32, 64) + 0.5    # shifted target domain

# MMD loss measures distribution distance
mmd = MMDLoss(kernel_type="rbf")
mmd_same   = mmd(source_features, source_features)
mmd_cross  = mmd(source_features, target_features)
print(f"MMD (source vs source): {mmd_same.item():.4f}  (should be ~0)")
print(f"MMD (source vs target): {mmd_cross.item():.4f}  (should be > 0)")

# Gradient Reversal Layer -- data passes through unchanged, gradients flip
grl = GradientReversalLayer(alpha=1.0)
x = torch.randn(4, 64, requires_grad=True)
y = grl(x)
print(f"\nGRL forward:  input == output? {torch.allclose(x, y)}")

# Domain discriminator
disc = DomainDiscriminator(input_dim=64, hidden_dim=32)
domain_logits = disc(source_features[:4])
print(f"Discriminator output: {tuple(domain_logits.shape)}")  # (4, 2)

<a id="13"></a>
## 13. Continual Learning (EWC)

Elastic Weight Consolidation prevents catastrophic forgetting when training on a sequence of tasks. It penalizes changes to parameters that were important for earlier tasks, as measured by the Fisher Information Matrix. A `ReplayBuffer` stores samples from previous tasks for experience rehearsal.

In [ ]:
from src.training.continual import EWCCallback, ReplayBuffer

# EWC callback -- would be used with PyTorch Lightning Trainer
ewc = EWCCallback(ewc_lambda=0.4)
print(f"EWC lambda: {ewc.ewc_lambda}")
print(f"Stored tasks: {len(ewc.fisher_matrices)}")

# Replay buffer for experience rehearsal
buf = ReplayBuffer(capacity=100)
buf.add_samples(list(range(50)))
buf.add_samples(list(range(50, 120)))  # exceeds capacity -> keeps last 100
print(f"\nReplay buffer size: {len(buf.buffer)} (capacity={buf.capacity})")
print(f"Sample 5 items:     {buf.sample(5)}")

<a id="14"></a>
## 14. Federated Learning

`FederatedAggregator` and `FederatedClient` implement FedAvg for privacy-preserving distributed training. Each client trains locally; the server aggregates updates weighted by dataset size.

In [ ]:
from src.training.federated import FederatedAggregator, FederatedClient

# Create a small global model
global_model = torch.nn.Linear(8, 2)

# Server
server = FederatedAggregator(global_model)

# Simulate 3 clients with local training
clients = [FederatedClient(torch.nn.Linear(8, 2), client_id=f"hospital_{i}") for i in range(3)]
dataset_sizes = [500, 300, 200]

for client, n_samples in zip(clients, dataset_sizes):
    # Pull latest global weights
    client.pull_global_weights(global_model.state_dict())
    # Simulate local training by adding noise to weights
    with torch.no_grad():
        for p in client.model.parameters():
            p.add_(torch.randn_like(p) * 0.1)
    # Send update back to server
    server.register_client_update(client.get_local_update(), n_samples)

# Aggregate using FedAvg
new_global = server.aggregate()
print("FedAvg aggregation complete.")
print(f"Global weight sample: {list(new_global.values())[0][0, :4].tolist()}")
print(f"Pending client updates: {len(server.client_weights)} (cleared after aggregation)")

<a id="15"></a>
## 15. Fairness Auditing

`FairnessAuditor` measures bias across sensitive attributes using three standard metrics:

| Metric | Ideal Value | Interpretation |
|---|---|---|
| Demographic Parity Difference | 0.0 | Max gap in positive-prediction rate between groups |
| Disparate Impact Ratio | 1.0 (> 0.8 is the "80 % rule") | Ratio of selection rates between least/most favored group |
| Equalized Odds Difference | 0.0 | Max gap in TPR or FPR between groups |

In [ ]:
from src.training.fairness import FairnessAuditor

# Synthetic predictions and sensitive attributes
torch.manual_seed(42)
y_true     = torch.randint(0, 2, (200,))
y_pred     = torch.randint(0, 2, (200,))
# Two demographic groups: 0 and 1
sensitive   = torch.cat([torch.zeros(100, dtype=torch.long), torch.ones(100, dtype=torch.long)])

auditor = FairnessAuditor()
report = auditor.audit(y_true, y_pred, sensitive)

print("Fairness Audit Report:")
for metric, value in report.items():
    print(f"  {metric:30s} = {value:.4f}")

<a id="16"></a>
## 16. Explainability

`ExplainabilityModule` provides Integrated Gradients for input attribution. Given a model and an input, it computes how much each input feature contributed to a target class prediction.

In [ ]:
from src.training.explainability import ExplainabilityModule

# Small classifier for demonstration
small_model = torch.nn.Sequential(
    torch.nn.Linear(8, 32),
    torch.nn.ReLU(),
    torch.nn.Linear(32, 3),  # 3 classes
)

sample_input = torch.randn(1, 8)
target_class = 1

attributions = ExplainabilityModule.integrated_gradients(
    model=small_model,
    inputs=sample_input,
    target_idx=target_class,
    steps=30,
)

print(f"Input shape:       {tuple(sample_input.shape)}")
print(f"Attribution shape:  {tuple(attributions.shape)}")
print(f"Top contributing features (by abs attribution):")
abs_attr = attributions.abs().squeeze()
top_k = torch.topk(abs_attr, k=3)
for idx, val in zip(top_k.indices.tolist(), top_k.values.tolist()):
    print(f"  feature {idx}: {val:.4f}")

<a id="17"></a>
## 17. Sidecar & IPC Architecture

The ML Zoo can run as a **Python sidecar** to a Rust/React host application. Communication happens over **NDJSON** (newline-delimited JSON) on stdin/stdout.

```
┌─────────────┐    stdin (NDJSON)    ┌──────────────────────┐
│  Rust / TS   │ ──────────────────▶ │  NdjsonTransport     │
│  Host App    │                     │  ↓                   │
│              │ ◀────────────────── │  MlRequestHandler    │
└─────────────┘   stdout (NDJSON)    │  ├─ inference.*      │
                                     │  ├─ model.*          │
                                     │  ├─ training.*       │
                                     │  ├─ device.*         │
                                     │  └─ voice.*          │
                                     └──────────────────────┘
```

**Supported RPC methods** (dispatched by `MlRequestHandler`):

| Namespace | Methods |
|-----------|---------|
| `health` | `ping` |
| `inference` | `complete`, `complete_stream`, `embed`, `plan`, `load_model` |
| `model` | `list`, `load`, `unload`, `download`, `migrate` |
| `training` | `start`, `stop`, `status`, `list`, `deploy`, `predict` |
| `device` | `info`, `refresh` |
| `voice` | `synthesize`, `transcribe` |
| `personality` | `hatch_chat` |

The entry point is `src/ml_sidecar_main.py`, which wires `NdjsonTransport` &#8594; `MlRequestHandler` &#8594; `InferenceEngine` / `ModelRegistry` / `DeviceManager`.

<a id="18"></a>
## 18. Configuration with Hydra / OmegaConf

The Zoo uses Hydra-style YAML configs organised into groups under `configs/`. At runtime, `OmegaConf` merges defaults with overrides and `deep_sanitize` converts them to plain Python dicts for PyTorch Lightning.

In [ ]:
from omegaconf import OmegaConf
from src.utils.config import deep_sanitize

# Configs are plain YAML dicts -- here we create one inline
cfg = OmegaConf.create({
    "model": {
        "backbone": "transformer",
        "head": "classification",
        "backbone_config": {"hidden_dim": 128, "num_layers": 4, "num_heads": 4},
        "head_config": {"num_classes": 10},
    },
    "training": {
        "learning_rate": 3e-4,
        "max_epochs": 20,
        "batch_size": 64,
    },
    "seed": 42,
})

# deep_sanitize converts OmegaConf containers to plain dicts / lists
plain = deep_sanitize(cfg)
print("Sanitised config (plain Python dict):")
for section, values in plain.items():
    print(f"  {section}: {values}")

<a id="19"></a>
## 19. Project Architecture

```
Machine-Learning-Zoo/
├── src/
│   ├── main.py                  # CLI entry point (build & demo models)
│   ├── ml_sidecar_main.py       # Sidecar server (NDJSON IPC)
│   │
│   ├── models/                  # All model definitions
│   │   ├── backbones/           #   Feature extractors (Transformer, LSTM, Mamba, Conv, HF, Vision, MultiModal)
│   │   ├── heads/               #   Task heads (Classification, Regression, RLPolicy, Sequence, MultiAgentPolicy)
│   │   ├── composed.py          #   build_model() factory
│   │   ├── registry.py          #   get_model() + MODEL_REGISTRY
│   │   ├── neuro_symbolic.py    #   Hybrid neural-symbolic architectures
│   │   ├── autoencoders/        #   VAE, SAE, Denoising AE, Stacked AE
│   │   ├── probabilistic/       #   RBM, DBN, Flows, Diffusion, GAN, Hopfield, Markov
│   │   ├── convolutional/       #   CNN, ResNet, Capsule, DeConv
│   │   ├── recurrent/           #   LSTM, GRU, xLSTM, Mamba, Echo State, Liquid State
│   │   ├── attention/           #   Transformer variants
│   │   ├── spiking/             #   SNN with LIF cells
│   │   ├── graphs/              #   GNN layers
│   │   ├── memory/              #   NTM, DNC
│   │   ├── competitive/         #   SOM, LVQ
│   │   ├── general/             #   MLP, RBF, ELM, PINN, NODE
│   │   ├── ensemble.py          #   Ensemble strategies
│   │   ├── mac/                 #   Classical ML (linear, trees, SVM, kNN, Bayes, ensemble)
│   │   └── helper/              #   Clustering, dim-reduction, association rules
│   │
│   ├── training/                # Training infrastructure
│   │   ├── trainer.py           #   ProgressCallback for Lightning
│   │   ├── lightning_module.py  #   PiLightningModule (LoRA, QLoRA, distillation)
│   │   ├── data_module.py       #   PiDataModule
│   │   ├── evaluation.py        #   Unified metrics
│   │   ├── explainability.py    #   Integrated Gradients, attention maps
│   │   ├── fairness.py          #   Bias auditing
│   │   ├── domain_adaptation.py #   GRL, MMD, DANN
│   │   ├── continual.py         #   EWC, replay buffer
│   │   ├── federated.py         #   FedAvg client / server
│   │   ├── multi_agent_rl.py    #   MAPPO trainer
│   │   └── automl.py            #   Optuna-based HPO
│   │
│   ├── inference/               # Multi-provider inference engine
│   │   ├── engine.py            #   InferenceEngine (local, Anthropic, Google, Deepseek)
│   │   ├── completion.py        #   Token generation
│   │   └── embeddings.py        #   Sentence embeddings
│   │
│   ├── envs/                    # Gymnasium environments
│   │   ├── base.py              #   EnvironmentProtocol, TradingEnvBase
│   │   ├── envs.py              #   TradingEnv, ClobEnv, PolymarketEnv
│   │   └── multi_agent.py       #   CooperativeGatheringEnv, CompetitiveArenaEnv
│   │
│   ├── pipeline/                # Training pipelines
│   │   ├── core/                #   Supervised, Unsupervised, SSL, GAN, Diffusion, RL (PPO / SAC)
│   │   ├── hpo/                 #   DEHB, Ray Tune, Optuna
│   │   ├── meta/                #   MAML, regime detection
│   │   ├── online_learning/     #   Online trainer, drift detection
│   │   └── active_learning/     #   Uncertainty & variance-reduction sampling
│   │
│   ├── features/                # Feature engineering
│   │   ├── pipeline.py          #   FeaturePipeline (scaling, selection, regime detection)
│   │   ├── normalization.py     #   OnlineNormalizer (Welford's algorithm)
│   │   └── regime.py            #   MarketRegimeDetector (HMM)
│   │
│   ├── device/                  # Hardware management
│   │   └── manager.py           #   DeviceManager (CPU, CUDA, MPS probing)
│   │
│   ├── ipc/                     # Inter-process communication
│   │   └── ndjson_transport.py  #   Async NDJSON over stdin/stdout
│   │
│   ├── configs/                 # Hydra / OmegaConf configuration
│   │
│   └── utils/                   # Utilities
│       ├── registry.py          #   Generic Registry[T], global registries
│       ├── config.py            #   deep_sanitize, sanitize_and_inject
│       ├── profiling/           #   Benchmarking, GPU optimisation
│       ├── io/                  #   MLflow, cloud storage, model versioning
│       ├── security/            #   Secrets management
│       └── export/              #   ONNX export
│
├── configs/                     # YAML config files
├── tests/                       # Pytest suite (unit, integration, property)
├── notebooks/                   # This notebook
├── examples/                    # Standalone example scripts
└── benchmark/                   # Performance benchmarks
```

<a id="20"></a>
## 20. Next Steps & Resources

**Where to go from here:**

- **ROADMAP.md** -- Planned features across 9 phases (foundation through community growth).
- **examples/** -- Standalone scripts demonstrating dashboards, training workflows, and inference.
- **tests/** -- The test suite doubles as executable documentation. Run with `pytest tests/`.
- **configs/** -- Browse the YAML config groups (`model/`, `algorithm/`, `env/`, `data/`) for ready-to-use presets.
- **CLI** -- `python src/main.py --list-presets` to see available model configs; `python src/main.py configs/model/transformer.yaml --demo` to run a quick demo.

**Key design principles:**
1. **Backbone + Head composition** -- Mix and match any backbone with any head via `build_model()`.
2. **Registry-driven** -- Every component registers itself; new models integrate by adding a single decorated class.
3. **PyTorch Lightning** -- Training pipelines use Lightning for distributed training, logging, and callbacks.
4. **Sidecar-ready** -- The entire library can serve as a subprocess to a host application via NDJSON IPC.